In [ ]:
n = 5
addr = 'C:\\Users\\lfox\\Downloads\\dks.csv'
# addr should point to csv including cols "Points", "Salary", "Cap", "CapVar", "team1", "team2", "wr", "rb", "qb"

max_num_team1 = 4
min_num_team1 = 4

max_num_team2 = 2
min_num_team2 = 2

max_num_wr = 3
min_num_wr = 2

max_num_rb = 3
min_num_rb = 1

max_num_qb = 2
min_num_qb = 1

In [ ]:
from pulp import *
import numpy as np
import pandas as pd
model = LpProblem('NFL',LpMaximize)

players = pd.read_csv(addr) 

results = []
total_points = {}
cost = {}
number_of_players = {}
cap = {}
cap_var = {}
team1 = {}
team2 = {}
wr = {}
qb = {}
rb = {}
x = {}

for i, player in players.iterrows():
    var_name = 'x' + str(i) # Create variable name
    decision_var = LpVariable(var_name, cat='Binary') # Initialize Variables
    
    x[i] = decision_var 
    
    total_points[decision_var] = player["Points"] # Create PPG Dictionary
    cost[decision_var] = player["Salary"] # Create Cost Dictionary
    
    cap[decision_var] = player["Cap"]
    cap_var[decision_var] = player["CapVar"]
    
    team1[decision_var] = player["team1"]
    team2[decision_var] = player["team2"]
    wr[decision_var] = player["wr"]
    rb[decision_var] = player["rb"]
    qb[decision_var] = player["qb"]

    number_of_players[decision_var] = 1.0
    
objective_function = LpAffineExpression(total_points)
total_cost = LpAffineExpression(cost)
total_players = LpAffineExpression(number_of_players)

cap_var_eq = LpAffineExpression(cap_var)
captain = LpAffineExpression(cap)

team1_eq = LpAffineExpression(team1)
team2_eq = LpAffineExpression(team2)
wr_eq = LpAffineExpression(wr)
rb_eq = LpAffineExpression(rb)
qb_eq = LpAffineExpression(qb)

model += objective_function
model += (total_cost <= 50000)
model += (total_players <= 6)
model += (captain >= 1)
model += (captain <= 1)
model += (total_players >= 6)
model += (cap_var_eq >= 1)
model += (cap_var_eq <= 1)
model += (team1_eq >= min_num_team1)
model += (team1_eq <= max_num_team1)
model += (team2_eq >= min_num_team2)
model += (team2_eq <= max_num_team2)
model += (wr_eq <= max_num_wr)
model += (wr_eq >= min_num_wr)
model += (rb_eq >= min_num_rb)
model += (rb_eq <= max_num_rb)
model += (qb_eq <= max_num_qb)
model += (qb_eq >= min_num_qb)

model.solve()

while len(results)<n:
    if model.solve() ==1:
        selection = []
        high = model.objective.value()
        players["is_drafted"] = 0.0
        for var in x.values():
            players.iloc[int(var.name[1:]),(len(players.columns)-1)] = var.varValue
        
        selection = players[players["is_drafted"] == 1.0]['ID']
            
        results.append(selection.values)
        
        model += (total_cost <= 50000)
        model += (total_players <= 6)
        model += (captain >= 1)
        model += (captain <= 1)
        model += (total_players >= 6)
        model += (cap_var_eq >= 1)
        model += (cap_var_eq <= 1)
        model += (team1_eq >= min_num_team1)
        model += (team1_eq <= max_num_team1)
        model += (team2_eq >= min_num_team2)
        model += (team2_eq <= max_num_team2)
        model += (wr_eq <= max_num_wr)
        model += (wr_eq >= min_num_wr)
        model += (rb_eq >= min_num_rb)
        model += (rb_eq <= max_num_rb)
        model += (qb_eq <= max_num_qb)
        model += (qb_eq >= min_num_qb)

        model += (objective_function <= (high - 0.01))
        
        model.solve()
        
for i in results:
    print(i)
    